In [1]:
#Import neccessary packages
from Bio import SeqIO
import pandas as pd
from Bio.SeqUtils import GC

In [6]:
#Defining a function to count the type and values of seq_record "types" in a GenBank file.
def feature_type_counter(seq_record):
    feature_types = {}
    for feature in seq_record.features:
        if feature.type in feature_types:
            feature_types[feature.type] += 1
        else:
            feature_types[feature.type] = 1

    for key, value in feature_types.items():
        print(f"{key}: {value}")

In [6]:
#Defining a class "sequence" with these attributes to make the analysis process easier.
class sequence:
    def __init__(self,seq_id,seq,seq_product,seq_translation,cog,start_location,end_location,strand_type,gene_name,gene_product,protein_sequence):
        self.id = seq_id
        self.seq = seq
        self.product =seq_product
        self.translation = seq_translation
        self.cog = cog
        self.startloc = start_location
        self.endloc = end_location
        self.strand = strand_type
        self.gene = gene_name
        self.product = gene_product
        self.translation = protein_sequence

In [8]:
#Defining a function that extracts the necessary information into a list.
def info_extraction(seq_record,feature_type):
    sequence_list = []
    for feature in seq_record.features:
        if feature.type == feature_type:       
            feature_id = feature.qualifiers['locus_tag'][0]
            feature_product = feature.qualifiers['product'][0]
            feature_sequence = feature.location.extract(seq_record).seq
            start_location = feature.location.start
            end_location = feature.location.end
            strand_type = feature.location.strand
            
            try:
              feature_translation = feature.qualifiers['translation'][0]
            except:
              feature_translation = "-"
            try:
              gene_name = feature.qualifiers['gene']
            except:
              gene_name = "-"
            try:
              gene_product = feature.qualifiers['product']
            except:
              gene_product = "-"
            try:
              protein_sequence = feature.qualifiers['translation']
            except:
              protein_sequence = "-"
            try:
                if feature.qualifiers['db_xref'][0].split(":")[0] == "COG":
                    feature_cog = feature.qualifiers['db_xref'][0].split(":")[1]
            except:
                feature_cog = "-" 

            sequence_list.append(sequence(feature_id,feature_sequence,feature_product,feature_translation,feature_cog,start_location,end_location,strand_type,gene_name,gene_product,protein_sequence))
    return sequence_list

In [9]:
#Defining a function that appends the extracted information from the GenBank file into the created lists.
def append_information(sequence_list):
    locus_tag_list = []
    start_location_list = []
    end_location_list = []
    strand_type_list = []
    gene_name_list = []
    gene_product_list = []
    nucleotide_sequence_list = []
    protein_sequence_list = []

    i = 0
    for sequence in sequence_list:
        locus_tag = sequence_list[i].id
        start_location = sequence_list[i].startloc
        end_location = sequence_list[i].endloc
        strand_type = sequence_list[i].strand
        gene_name = sequence_list[i].gene
        gene_product = sequence_list[i].product
        nucleotide_sequence = sequence_list[i].seq
        protein_sequence = sequence_list[i].translation

        
        locus_tag_list.append(locus_tag)
        start_location_list.append(start_location)
        end_location_list.append(end_location)
        strand_type_list.append(strand_type)
        gene_name_list.append(gene_name)
        gene_product_list.append(gene_product)
        nucleotide_sequence_list.append(nucleotide_sequence)
        protein_sequence_list.append(protein_sequence)

        i += 1
    return locus_tag_list, start_location_list, end_location_list, strand_type_list,gene_name_list,gene_product_list,nucleotide_sequence_list,protein_sequence_list

In [ ]:
#Creating the tRNA counting function
def tRNA_counter():
    tRNA_list = []

    for i in range(gbk_df.shape[0]):
        tRNA_raw = gbk_df["product"][i][0]
        tRNA_cleaned = tRNA_raw[5:]
        tRNA_list.append(tRNA_cleaned)

    tRNA_dict = dict(Counter(tRNA_list))
    tRNA_df = pd.DataFrame(tRNA_dict.items(), columns=['Jenis tRNA','Jumlah'])
    return tRNA_df

In [43]:
#Opening the genbank file
gbk_directory = "/path/to/genome.gbk"
seq_record_list = []
for seq_record in SeqIO.parse(gbk_directory, "genbank"):
    seq_record_list.append(seq_record)

In [ ]:
#Counting the number of contig contained in the genbank file. 
number_of_contig_sequence = len(seq_record_list)
print("Jumlah contig:",number_of_contig_sequence)

In [ ]:
#Counting the genome length and the GC percentage.
genome_sequence = ''
for i in range(len(seq_record_list)):
    genome_sequence+=seq_record_list[i].seq
    
genome_length = len(genome_sequence)
genome_GCpercent = GC(genome_sequence)

print("Ukuran genom:",genome_length)
print("GC-percentage:",genome_GCpercent)

In [ ]:
#Searching for the available number of "types" in the GenBank file
for seq_record in seq_record_list:
    feature_type_counter(seq_record)
    print("-----")

In [ ]:
#Extracting the information needed from the GenBank file using the previously defined function: "info_extraction()" and append it into the created list using the previously defined function: "append_information()"
feature_type = 'CDS'

final_locus_tag_list = []
final_start_location_list = []
final_end_location_list = []
final_strand_type_list = []
final_gene_name_list = []
final_gene_product_list = []
final_nucleotide_sequence_list = []
final_protein_sequence_list = []

for seq_record in seq_record_list:
    sequence_list = info_extraction(seq_record,feature_type)
    locus_tag_list, start_location_list, end_location_list, strand_type_list, gene_name_list, gene_product_list, nucleotide_sequence_list, protein_sequence_list = append_information(sequence_list)
    final_locus_tag_list += locus_tag_list
    final_start_location_list += start_location_list
    final_end_location_list += end_location_list
    final_strand_type_list += strand_type_list
    final_gene_name_list += gene_name_list
    final_gene_product_list += gene_product_list
    final_nucleotide_sequence_list += nucleotide_sequence_list
    final_protein_sequence_list += protein_sequence_list

In [ ]:
#Creating data frame from the created list
gbk_df = pd.DataFrame()

name_column = pd.Series(final_locus_tag_list, name="name")
start_column = pd.Series(final_start_location_list, name="start")
stop_column = pd.Series(final_end_location_list, name="stop")
strand_column = pd.Series(final_strand_type_list, name="strand")
gene_column = pd.Series(final_gene_name_list, name="gene")
product_column = pd.Series(final_gene_product_list, name="product")
sequence_column = pd.Series(final_nucleotide_sequence_list, name="sequence")
translation_column = pd.Series(final_protein_sequence_list, name="translation")

c1 = name_column.to_frame()
c2 = start_column.to_frame()
c3 = stop_column.to_frame()
c4 = strand_column.to_frame()
c5 = gene_column.to_frame()
c6 = product_column.to_frame()
c7 = sequence_column.to_frame()
c8 = translation_column.to_frame()

gbk_df = pd.concat([c1,c2,c3,c4,c5,c6,c7,c8], axis=1)
gbk_df['strand'] = gbk_df['strand'].replace(1, '+')
gbk_df['strand'] = gbk_df['strand'].replace(-1, '-')
gbk_df['start'] += 1

gbk_df


In [54]:
#Saving the data frame into a CSV file
gbk_df.to_csv("/nama_file.csv",index=False)

In [ ]:
#Counting the tRNA contained in the genome
tRNA_df = tRNA_counter()
tRNA_df
tRNA_df.to_csv("path/ke/file/.csv",index=False)